In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# You can check the column names from the readme file

In [3]:
#Reading files:
user = pd.read_csv('user_data.csv', encoding='latin-1')
problems = pd.read_csv('problem_data.csv', encoding='latin-1')
attempts = pd.read_csv('train_submissions.csv', encoding='latin-1')
test = pd.read_csv('test_submissions.csv', encoding='latin-1')

In [4]:
#print(user)
print(user.shape)
user.head()

(3571, 11)


,user_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_3311,47,40,0,NaN,4,1504111645,348.337,330.849,intermediate,1466686436
1,user_3028,63,52,0,India,17,1498998165,405.677,339.450,intermediate,1441893325
2,user_2268,226,203,-8,Egypt,24,1505566052,307.339,284.404,beginner,1454267603
3,user_480,611,490,1,Ukraine,94,1505257499,525.803,471.330,advanced,1350720417
4,user_650,504,479,12,Russia,4,1496613433,548.739,486.525,advanced,1395560498


In [5]:
print(problems.shape)
problems.head()

(6544, 4)


,problem_id,level_type,points,tags
0,prob_3649,H,NaN,NaN
1,prob_6191,A,NaN,NaN
2,prob_2020,F,NaN,NaN
3,prob_313,A,500.0,"greedy,implementation"
4,prob_101,A,500.0,"constructive algorithms,greedy,math"


In [6]:
print(attempts.shape)
attempts.head()

(155295, 3)


,user_id,problem_id,attempts_range
0,user_232,prob_6507,1
1,user_3568,prob_2994,3
2,user_1600,prob_5071,1
3,user_2256,prob_703,1
4,user_2321,prob_356,1


In [7]:
print(test.shape)

(66555, 3)


In [8]:
n_users = attempts.user_id.unique().shape[0]

n_items = attempts.problem_id.unique().shape[0]
print(n_users)
print(n_items)

3529
5776


In [9]:
us=[]
empt=[]
for i in attempts.user_id:
    us=int(i.split('_')[1]) 
    empt.append(us)
attempts.user_id=empt
#print(attempts.user_id)
us=[]
empt=[]
for i in attempts.problem_id:
    us=int(i.split('_')[1]) 
    empt.append(us)
attempts.problem_id=empt
x=attempts.user_id.unique()
y=attempts.problem_id.unique()
data_matrix = np.zeros((n_users, n_items))
for line in attempts.itertuples():
    data_matrix[x.tolist().index(line[1]),y.tolist().index(line[2])] = line[3]
print(data_matrix)
    

[[1. 0. 4. ... 0. 0. 0.]
 [0. 3. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
problem_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [11]:
def predict(attempts, similarity, type='user'):
    if type == 'user':
        mean_user_attempts = attempts.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        attempts_diff = (attempts - mean_user_attempts[:, np.newaxis])
        pred = mean_user_attempts[:, np.newaxis] + similarity.dot(attempts_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'problem':
        pred = attempts.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [12]:
user_prediction = predict(data_matrix, user_similarity, type='user')
problem_prediction = predict(data_matrix, problem_similarity, type='problem')

In [13]:
print(user_prediction)
#print(problem_prediction)

[[ 0.00485579  0.05576887  0.57314487 ... -0.00284706 -0.00282466
  -0.00311772]
 [ 0.01249795  0.05985917  0.61257885 ...  0.00451231  0.00455256
   0.00425482]
 [ 0.00307646  0.05404908  0.5960476  ... -0.00505643 -0.00500584
  -0.00530963]
 ...
 [-0.00461869  0.04654265  0.59828611 ... -0.01268373 -0.01268373
  -0.01297403]
 [-0.00432817  0.04654264  0.60498342 ... -0.01228301 -0.01228301
  -0.01256711]
 [-0.00472109  0.04687586  0.59939226 ... -0.0126715  -0.0126715
  -0.01296131]]


In [16]:
class MF():

    # Initializing the user-problems attempts matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and problem-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and problem j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-problem attempts matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [17]:
R= np.array(attempts.pivot(index = 'user_id', columns ='problem_id', values = 'attempts_range').fillna(0))

In [18]:
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=100)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 374.3966
Iteration: 40 ; error = 366.1958
Iteration: 60 ; error = 361.3474
Iteration: 80 ; error = 356.4470
Iteration: 100 ; error = 349.8692
()
P x Q:
[[1.1306516  1.09545718 2.01010804 ... 1.18856061 1.12612163 1.07420513]
 [2.17708979 2.18355335 3.04313713 ... 2.21261579 2.20376911 2.14872911]
 [1.63586328 1.61426893 2.48537013 ... 1.66135752 1.63778361 1.57482155]
 ...
 [1.3922099  1.34914796 2.30022161 ... 1.42609212 1.37142129 1.30749344]
 [1.88564125 1.8419202  2.84307772 ... 1.94656738 1.92181596 1.8477513 ]
 [1.42935232 1.40308827 2.3961787  ... 1.46587663 1.47461825 1.37514881]]
()
